In [1]:
from MDSetup import MDSetup

# MDSetup

This notebook enables the setup and analysis of MD simulations using LAMMPS in an automated fashion. The idea is to introduce the **FAIR** principles:

1. **F**indable ( metadata and data should be easy to find for humans and computers )
2. **A**ccessible ( (meta)data should accessable by standardised communications protocol, e.g.: file formats )
3. **I**nteroperable ( data need to interoperate with applications or workflows for analysis, storage, and processing )
4. **R**eusable ( (meta)data should be well-described to replicate it in different settings )

There is the possiblity to provide data and LAMMPS compatible parameter files, or to build a system and write all necessary input using LAMMPS and moleculegraph.

1) Read in the YAML files to define the system and simulation/sampling settings.

In [2]:
lammps_setup = MDSetup( system_setup = "input/lammps/setup.yaml", 
                        simulation_default = "input/lammps/defaults.yaml",
                        simulation_ensemble = "input/lammps/ensemble.yaml",
                        simulation_sampling = "input/lammps/sampling.yaml",
                        submission_command = "qsub"
                     )

MD input will be generated for 'lammps'!


## Creating force field and molecule files

In this section, MDSetup and moleculegraph (https://github.com/maxfleck/moleculegraph) is used to setup a force field and molecule files, which are needed to build and simulate the system

In [12]:
lammps_setup.write_topology( verbose = True )


Utilize moleculegraph to generate molecule and topology files of every molecule in the system!

Force field provided for software 'lammps'

Matching atoms to force field for SMILES: CCC

Atom 0 (C) is matched to ff type: CH3_alkane
Atom 1 (C) is matched to ff type: CH2_alkane
Atom 2 (C) is matched to ff type: CH3_alkane
Atom 3 (H) is matched to ff type: carbon_h
Atom 4 (H) is matched to ff type: carbon_h
Atom 5 (H) is matched to ff type: carbon_h
Atom 6 (H) is matched to ff type: carbon_h
Atom 7 (H) is matched to ff type: carbon_h
Atom 8 (H) is matched to ff type: carbon_h
Atom 9 (H) is matched to ff type: carbon_h
Atom 10 (H) is matched to ff type: carbon_h

Final molecule after removing carbon h's

Atom 0 (C): CH3_alkane
Atom 1 (C): CH2_alkane
Atom 2 (C): CH3_alkane

Matching atoms to force field for SMILES: C=C

Atom 0 (C) is matched to ff type: CH2_alkene
Atom 1 (C) is matched to ff type: CH2_alkene
Atom 2 (H) is matched to ff type: carbon_h
Atom 3 (H) is matched to ff type: carbo

## Setting up a simulation pipeline

In this section the possibility to setup a simulation folder, along with a simulation pipeline using several ensembles, is provided.

1) Setup simulation and build initial system (if not provided)


In [13]:
# Define the simulation folder
simulation_folder = "md_thermo"

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = [ "em", "npt" ] 

# Define the simulation time per ensemble in nano seconds (for em the number of iterations is provided in the ensemble yaml)
simulation_times = [ 0, 0.1 ]

# Define initial systems, in case the simulation should be continued from a prior simulation.
# In that case, provide one initial structure for each temperature & pressure state.
# If the simulation should start from an initial configuration, provide an empty list.
initial_systems = [ ]

# Provide kwargs that should be passed into the input template directly
input_kwargs = {  }

# Define number of copies
copies = 1

# Define if the inital system should build locally or with the cluster
on_cluster = False

# Define the starting number for the first ensemble ( 0{off_set}_ensemble )
off_set    = 0

# In case templates for job submission or any other templates are edited
# Any keyword argument can be passed to this function call
lammps_setup.prepare_simulation( 
    folder_name = simulation_folder,
    ensembles = ensembles,
    simulation_times = simulation_times,
    initial_systems = initial_systems,
    input_kwargs = input_kwargs,
    copies = copies,
    on_cluster = on_cluster,
    off_set = off_set 
)


Building system based on provided molecule numbers and coordinate files!


Build system locally! Wait until it is finished.

Build successful



2) Submit jobs to cluster

In [9]:
# Submit the simulations
lammps_setup.submit_simulation()


Submitting simulations at temperature: 300.0 K, pressure: 1.0 bar, density: 560.0 kg/m^3, fraction: 0.2 mol/mol.

Submitting job: /beegfs/work/st_ac137577/software/MDSetup/example/simulation_setup/input/lammps/../../lammps/butane_hexane/md_thermo/temp_300.0_pres_1.0_dens_560.0_frac_0.2/copy_0/job_300_1.sh
11592135


Submitting job: /beegfs/work/st_ac137577/software/MDSetup/example/simulation_setup/input/lammps/../../lammps/butane_hexane/md_thermo/temp_300.0_pres_1.0_dens_560.0_frac_0.2/copy_1/job_300_1.sh
11592136




## Extract sampled properties

In [7]:
# Extract properties from LAMMPS and analyse them

# Define analysis folder
analysis_folder = "md_thermo"

# Define analysis ensemble
ensemble = "01_npt"  

# Properties to extract
properties = ["temperature", "potential energy", "kinetic energy", "enthalpy"]

# Suffix of output file
output_suffix = "energy"

# Percentage to discard from beginning of the simulation
time_fraction = 0.1

lammps_setup.analysis_extract_properties( 
    analysis_folder = analysis_folder, 
    ensemble = ensemble, 
    extracted_properties = properties, 
    output_suffix = output_suffix, 
    time_fraction = time_fraction 
)

temperature: 300.0 K, pressure: 1.0 bar, density: 560.0 kg/m^3, mole_fraction: 0.2 mol/mol
   /beegfs/work/st_ac137577/software/MDSetup/example/simulation_setup/input/lammps/../../lammps/butane_hexane/md_thermo/temp_300.0_pres_1.0_dens_560.0_mole_0.2/copy_0/01_npt/npt.energy
   /beegfs/work/st_ac137577/software/MDSetup/example/simulation_setup/input/lammps/../../lammps/butane_hexane/md_thermo/temp_300.0_pres_1.0_dens_560.0_mole_0.2/copy_1/01_npt/npt.energy


Averaged values over all copies:

            property        mean        std      unit
0       temperature  300.026250   1.366208         K
1  potential energy -603.631611  28.856931  kcal/mol
2    kinetic energy  803.994806   3.661073  kcal/mol
3          enthalpy  199.672178  27.687412  kcal/mol 

